# <center> INTRODUÇÃO A DATASCIENCE </center> 

#### Fernando A. Correia dos S. Junior <br> João Vitor Chaves de Oliveira

Árvore geradora mínima problema valorar as arestas <br>
Por que é necessário ser acíclico? O grafo? 

## ToDo List

- **E dos demais, quantos tem uma citação? e duas?  ## Só chamar out_degree ou in_degree passando 1,2 ...**
- **And after that?How many has more than 10 citations? And 100? **
- **Distribuição de graus, comparação por ano e com o total**
- **Distribuição de número de componentes, comparação por ano e com o total**
- Distribuição da média de nós por componentes, comparação por ano e com o total
- Comparar medidas de centralidade, escolher 3 casos e plotar a distribuição das métricas ao longo do tempo.
- **Hubs e Authorities** 

In [1]:
import networkx as nx
import json
import random
import re
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
class Graph():
    def __init__(self, path, graph_name="grafo-normalizado"):
        self.graph_name = graph_name
        self.GraphJSON = json.load(open(path)) 
        self.G = nx.DiGraph()
        self.edges = []
        self.generateGraph()

    def generateGraph(self): #Esta é a função que gera o grafo
        keys = self.GraphJSON.keys()
        for i in keys:
            values = self.GraphJSON[i]
            for j in values:
                self.G.add_edge(i,j)
        
    def getNumberOfEdges(self):
        return self.G.number_of_edges()

    def getNumberOfVertex(self):
        return self.G.number_of_nodes()
    
    def saveGraph(self):
        nx.write_graphml(self.G, path='%s.graphml' % self.graph_name)
        
    def centrality(self):
        centralities = nx.algorithms.degree_centrality(self.G)
        return centralities
    
    def betweenness_centrality(self):
        centralities = nx.algorithms.betweenness_centrality(self.G)
        return centralities
    
    def eigenvector_centrality(self):
        centralities = nx.algorithms.eigenvector_centrality_numpy(self.G)
        return centralities
    

    def in_degree(self, x): #Retorna a lista de vértices com grau de entrada x
        l = []
        for v in self.G.nodes():
            if self.G.in_degree(v) == x:
                l.append(v)
        return l
    
    def out_degree(self, x): #Retorna a lista de vértices com grau de saída x
        l = []
        for v in self.G.nodes():
            if self.G.out_degree(v) == x:
                l.append(v)
        return l
    
    def max_degree(self, arg): # arg = {int ou out} in para grau in e out para grau out
        maximum = -1
        if arg=='in':
            for v in self.G.nodes():
                degree = self.G.in_degree(v)
                if maximum < degree:
                    maximum = degree
        elif arg=='out':
            for v in self.G.nodes():
                degree = self.G.out_degree(v)
                if maximum < degree:
                    maximum = degree
        else:
            print(u"Entrada inválida")
        return maximum
    
    def nodes(self):
        return self.G.nodes
       
    def hubs_and_authorities(self):
        return nx.hits(self.G)
    
    def number_strongly_connected_component(self):
        return nx.number_strongly_connected_components(self.G)

    def strongly_connected_component(self):
        return sorted(nx.connected_components(self.G), key=len, reverse=True)
        
    def degree_distribuition(self, title_year = ''):
        din = self.max_degree('in')
        dout = self.max_degree('out')
        MaxDegree = max(din,dout)
        values = np.zeros([MaxDegree+1,2])
        
        for i in range(MaxDegree+1):
            values[i][0] = len(self.in_degree(i))
            values[i][1] = len(self.out_degree(i))
        df = pd.DataFrame(values,columns=['Grau de entrada','Grau de Saída'])
        p = df.plot.bar()
        p.set_xlabel(u'Grau')
        p.set_ylabel(u'Número de vértices')
        p.set_title(u"Número de vértices por grau "+title_year)
    
    def get_vertex_max_in_degree(self):
        maximum = -1
        max_node = None
        for v in self.G.nodes():
            degree = self.G.in_degree(v)
            if maximum < degree:
                maximum = degree
                max_node = v
        return max_node
            
    def list_in_degree(self):
        return [self.G.in_degree(v) for v in self.G.nodes()]
    
    def list_out_degree(self):
        return [self.G.out_degree(v) for v in self.G.nodes()]

    def showGraph(self): #plota o grafo mas não fica tão legal 
        nx.draw_networkx(self.G)
        plt.show()
        

In [3]:
windows = [
    (2005, Graph('normalizado_citation_map_aborto_2005.json')),
    (2006, Graph('normalizado_citation_map_aborto_2006.json')),
    (2007, Graph('normalizado_citation_map_aborto_2007.json')),
    (2008, Graph('normalizado_citation_map_aborto_2008.json')),
    (2009, Graph('normalizado_citation_map_aborto_2009.json')),
    (2010, Graph('normalizado_citation_map_aborto_2010.json')),
    (2011, Graph('normalizado_citation_map_aborto_2011.json')),
    (2012, Graph('normalizado_citation_map_aborto_2012.json')),
    (2013, Graph('normalizado_citation_map_aborto_2013.json')),
    (2014, Graph('normalizado_citation_map_aborto_2014.json')),
    (2015, Graph('normalizado_citation_map_aborto_2015.json')),
    (2016, Graph('normalizado_citation_map_aborto_2016.json')),
    
]

### Nós comuns a todas as janelas

In [4]:
nodes_list = [set(graph[1].nodes()) for graph in windows]
all_nodes = set.union(*nodes_list)
comumn_nodes = set.intersection(*nodes_list)
print(u"Total de nós comuns a todos os anos: %s" % len(comumn_nodes))
print(u"Total de nós: %s" % len(all_nodes))

Total de nós comuns a todos os anos: 0
Total de nós: 1287


### Nós comuns entre janelas de um ano

In [5]:
for i in range(len(windows) - 1):
    set_a = set(windows[i][1].nodes())
    set_b = set(windows[i + 1][1].nodes())
    inter = set_a & set_b
    print("++++++++++++++++++++++++++++++++++++++")
    print(u"Entre %s e %s há %s vertices em comum" % (windows[i][0], windows[i +1][0], len(inter)))
    print(inter)

++++++++++++++++++++++++++++++++++++++
Entre 2005 e 2006 há 8 vertices em comum
set([u'hc 70763 stf', u'hc 2 stj', u'com 3 stj', u'adpf 54 stf', u'ms 2 stj', u'inq 2 stf', u'adi 2 stf', u'hc 84768 stf'])
++++++++++++++++++++++++++++++++++++++
Entre 2006 e 2007 há 5 vertices em comum
set([u'hc 2 stj', u'adpf 54 stf', u'inq 2 stf', u'adi 1 stf', u'rhc 2 stj'])
++++++++++++++++++++++++++++++++++++++
Entre 2007 e 2008 há 9 vertices em comum
set([u'hc 72424 stf', u'adpf 79 stf', u'hc 2 stj', u'adpf 33 stf', u'hc 7368611 stf', u'se 8 stj', u'adpf 54 stf', u'hc 70193 stf', u'adi 3 stf'])
++++++++++++++++++++++++++++++++++++++
Entre 2008 e 2009 há 11 vertices em comum
set([u'hc 72424 stf', u'com 7 stj', u'com 4 stj', u'sum. 691 stf', u'hc 7368611 stf', u'adpf 54 stf', u'hc 70193 stf', u'adpf 33 stf', u'adpf 12 stf', u'adpf 13 stf', u'se 2 stj'])
++++++++++++++++++++++++++++++++++++++
Entre 2009 e 2010 há 19 vertices em comum
set([u're 381964 stf', u'adi 2675 stf', u'hc 72424 stf', u'adi 2223 s

### Listagem dos nós que "vivem" mais de 2 anos

In [27]:
commun_nodes = set()
for i in range(len(windows) - 1):
    set_a = set(windows[i][1].nodes())
    set_b = set(windows[i + 1][1].nodes())
    inter = set_a & set_b
    commun_nodes = commun_nodes | inter

print("Total de nós que vivem mais de 2 anos: %s " % len(commun_nodes))
print(commun_nodes)

Total de nós que vivem mais de 2 anos: 119 
set([u'hc 68219 stf', u'sta 312 stf', u're 631102 stf', u'adi 2777 stf', u'adi 3605 stf', u'ms 25594 stf', u'hc 73721 stf', u'rcl 497 stf', u'hc 70763 stf', u'inq 2 stf', u'hc 93596 stf', u'hc 100975 stf', u'se 12 stj', u'adi 1127 stf', u'sta 32484 stf', u'hc 84768 stf', u'adi 1946 stf', u'com 7 stj', u'sta 507 stf', u'adi 354 stf', u're 129392 stf', u'adi 2566 stf', u'hc 7368611 stf', u'hc 73686 stf', u'adpf 54 stf', u'adi 3685 stf', u'hc 135558 stj', u'rhc 2 stj', u'hc 113244 stf', u'sum. 734 stf', u'if 5129 stf', u'resp 574511 stj', u'com 4 stj', u'hc 66954 stf', u'sum. 279 stf', u'rhc 65127 stj', u'sum. 691 stf', u're 566471 stf', u're 405579 stf', u'mc 2130 stj', u'hc 13897 stj', u'hc 13526 stj', u'mc 2321 stj', u'hc 88707 stf', u'hc 67271 stf', u'adpf 13 stf', u'adi 3 stf', u'hc 48629 stj', u'se 8 stj', u'hc 111953 stf', u'adi 3741 stf', u'com 3 stj', u'rhc 93248 stf', u'adi 2 stf', u'adpf 33 stf', u'adi 4307 stf', u'sta 304 stf', u'adi